### Import library

In [1]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torchsummary import summary

import matplotlib.pyplot as plt
from PIL import Image
import time

### Download training data

In [2]:
!wget --no-check-certificate https://storage.googleapis.com/emcassavadata/cassavaleafdata.zip \
                -O /content/cassavaleafdata.zip
!unzip /content/cassavaleafdata.zip

Streaming output truncated to the last 5000 lines.
  inflating: cassavaleafdata/train/cgm/train-cgm-679.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-68.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-680.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-681.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-682.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-683.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-684.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-685.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-686.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-687.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-688.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-689.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-69.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-690.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-691.jpg  
  inflating: cassavaleafdata/train/cgm/train-cgm-692.jpg  
  infla

### Split training dataset

In [4]:
data_path = {
    'train': '/content/cassavaleafdata/train',
    'valid': '/content/cassavaleafdata/validation',
    'test': '/content/cassavaleafdata/test'
}

# load image from path


def loader(path):
    return Image.open(path)


img_size = 150
train_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])

train_data = datasets.ImageFolder(
    root=data_path['train'],
    loader=loader,
    transform=train_transforms
)

valid_data = datasets.ImageFolder(
    root=data_path['valid'],
    transform=train_transforms
)

test_data = datasets.ImageFolder(
    root=data_path['test'],
    transform=train_transforms
)

BATCH_SIZE = 512

train_dataloader = data.DataLoader(
    train_data,
    shuffle=True,
    batch_size=BATCH_SIZE
)
valid_dataloader = data.DataLoader(
    valid_data,
    batch_size=BATCH_SIZE
)
test_dataloader = data.DataLoader(
    test_data,
    batch_size=BATCH_SIZE
)


### Define model

In [5]:
class LeNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=6, kernel_size=5, padding='same'
        )
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc_1 = nn.Linear(16*35*35, 120)
        self.fc_2 = nn.Linear(120, 84)
        self.fc_3 = nn.Linear(84, num_classes)

    def forward(self, inputs):
        outputs = self.conv1(inputs)
        outputs = self.avgpool1(outputs)
        outputs = F.relu(outputs)
        outputs = self.conv2(outputs)
        outputs = self.avgpool2(outputs)
        outputs = F.relu(outputs)
        outputs = self.flatten(outputs)
        outputs = self.fc_1(outputs)
        outputs = self.fc_2(outputs)
        outputs = self.fc_3(outputs)
        return outputs


### Train function

In [6]:
def train(model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50):
    model.train()
    total_acc, total_count = 0, 0

    losses = []
    start_time = time.time()

    for idx, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        predictions = model(inputs)

        # compute loss
        loss = criterion(predictions, labels)
        losses.append(loss.item())

        # backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)

        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| Epoch {:3d} | {:5d}/{:5d} batches"
                "| Accuracy {:8.3f}".format(
                    epoch, idx, len(train_dataloader), total_acc/total_count
                ))

            total_acc, total_count = 0, 0
            start_time = time.time()

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss


### Evaluation function

In [7]:
def evaluate(model, criterion, valid_dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    losses = []
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            predictions = model(inputs)
            loss = criterion(predictions, labels)
            losses.append(loss.item())

            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)

    return epoch_acc, epoch_loss


### Training

In [10]:
# Training
num_classes = len(train_data.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lenet_model = LeNetClassifier(num_classes)
learning_rate = 2e-4
lenet_model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(lenet_model.parameters(), lr=learning_rate)

num_epochs = 10
save_model = '/content/drive/MyDrive/AIO_2024/Module_6/Week_1/Exercise_2_1_2/model'

train_accs, train_losses = [], []
eval_accs, eval_losses = [], []
best_loss_eval = 100
for epoch in range(1, num_epochs + 1):
    epoch_start_time = time.time()

    # training
    train_acc, train_loss = train(
        lenet_model, optimizer, criterion, train_dataloader, device, epoch)

    train_accs.append(train_acc)
    train_losses.append(train_loss)

    # evaluate
    eval_acc, eval_loss = evaluate(lenet_model, criterion, valid_dataloader)
    eval_accs.append(eval_acc)
    eval_losses.append(eval_loss)

    # save best model
    if eval_loss < best_loss_eval:
        torch.save(lenet_model.state_dict(), save_model + '/lenet_model.pt')

    # print loss, acc and epoch
    print("===========")
    print(
        "| End of epoch {:3d} | time: {:5.2f}s | train accuracy {:8.3f} | train loss {:8.3f}"
        "| Valid Accuracy {:8.3f} | Valid loss {:8.3f}".format(epoch, time.time() - epoch_start_time, train_acc, train_loss, eval_acc, eval_loss))
    print("===========")

    # load best model
    lenet_model.load_state_dict(torch.load(save_model + '/lenet_model.pt'))
    lenet_model.eval()


| End of epoch   1 | time: 120.17s | train accuracy    0.451 | train loss    1.401| Valid Accuracy    0.470 | Valid loss    1.363


<ipython-input-10-b1a14452f9a6>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lenet_model.load_state_dict(torch.load(save_model + '/lenet_model.pt'))


| End of epoch   2 | time: 117.74s | train accuracy    0.472 | train loss    1.334| Valid Accuracy    0.475 | Valid loss    1.314
| End of epoch   3 | time: 117.92s | train accuracy    0.484 | train loss    1.267| Valid Accuracy    0.520 | Valid loss    1.281
| End of epoch   4 | time: 117.06s | train accuracy    0.501 | train loss    1.258| Valid Accuracy    0.519 | Valid loss    1.253
| End of epoch   5 | time: 116.76s | train accuracy    0.510 | train loss    1.246| Valid Accuracy    0.525 | Valid loss    1.238
| End of epoch   6 | time: 118.09s | train accuracy    0.514 | train loss    1.230| Valid Accuracy    0.533 | Valid loss    1.236
| End of epoch   7 | time: 116.75s | train accuracy    0.522 | train loss    1.215| Valid Accuracy    0.538 | Valid loss    1.223
| End of epoch   8 | time: 117.02s | train accuracy    0.525 | train loss    1.204| Valid Accuracy    0.551 | Valid loss    1.218
| End of epoch   9 | time: 117.98s | train accuracy    0.540 | train loss    1.171| Valid 

### Evaluate model

In [11]:
test_acc, test_loss = evaluate(lenet_model, criterion, test_dataloader)
print(test_acc)
print(test_loss)

0.5458885941644562
1.1815250664949417
